# Generate centre manifold data
----
In this file we generate the centre manifold data which is then used to construct the immersion.

In [1]:
import numpy as np
from numpy import cos, sin, cosh, sinh, tanh, array,pi, exp,array,sqrt
from numpy.linalg import norm,solve
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import scipy.linalg
import scipy.io
from model import linearisedSystem,model,eigen,Usolve,Lsolve
h,k = tuple(np.load("ModelParameters.npy"))
D  =lambda v: k*v
ForcingParameters = tuple(np.load("ForcingParameters.npy"))
T,N = tuple(np.load("TimeInterval.npy"))
N = int(N)
forcingtypevector = np.load("forcingtype.npy")
if forcingtypevector[0] == 0:
    forcingtype = "periodic"
elif forcingtypevector[0] == 1:
    forcingtype = "quasi-periodic"
else:
    print("clarify forcingtype")
if   forcingtype == "periodic":
    Amp1,w1 = ForcingParameters
    G = lambda t : Amp1*cos(w1*t)        
elif forcingtype == "quasi-periodic":
    n = len(ForcingParameters)//3
    Amp = np.zeros(n)
    w = np.zeros(n)
    eps = np.zeros(n)
    for i in range(0,n):
        Amp[i] = ForcingParameters[3*i]
        w[i] = ForcingParameters[3*i+1]
        eps[i] = ForcingParameters[3*i+2]
    G = lambda t : Amp.dot(cos(t*w + eps))  


In [2]:
thyperbolic = np.linspace(-T,T,N)
f,Aplus, Aminus, xplus, xminus = model(h,k,D,G)
dt = 2*T/N
Pplus,Pplusinv,Pminus,Pinvminus,lambda2, omega, lambda3, lambda4  =  eigen(h,k)
Xhyperbolicplus= np.load('Xhyperbolicplus.npy')
Xhyperbolicminus= np.load('Xhyperbolicminus.npy')

## Generating data
-----
First we set the parameters that determine which trajectories are found. $R$ is the distance along the approximate eigenvectors that we search, Deltaminus and Deltaplus represent for how long we do integrate along the equation. The t0indices indicate at which times we calculate the stable manifold points, which are thyperbolic[t0indices] 

In [3]:
def FindCentrePoints(CapsizeSide):
    if CapsizeSide == "positive":
        Xhyperbolic = Xhyperbolicplus # The positive hyperbolic trajectory
        A = Aplus # Df at positive saddle point
        P = Pplus # eigen bcasis matrix for A
        Pinv =  Pplusinv # iverse of change of basis matrix 
    elif CapsizeSide == "negative":
        Xhyperbolic = Xhyperbolicminus #  negative hyperbolic trajectory
        A = Aminus # Df at negative saddle point
        P = Pminus # eigenbasis matirx for A
        Pinv =  Pinvminus # inverse change of basis matrix
    Dphi = scipy.linalg.expm(dt*A) # approximate derivative of flow
    def defineBVP(t0index,imax, imin):  # this sets up the bvp for the centre trajectory
        l = t0index - imin
        def g(X,q1,q2):
            # q1, q2 center coordinates
            Y = np.zeros((imax - imin+1,4))
            X = X.reshape((imax - imin+1  ,4)) # X is displacement of trajectory from the hyperbolic trajectory
            for i in range(0, imax -imin-1):
                Y[i] = phi(X[i]+Xhyperbolic[i + imin],thyperbolic[i+imin]) - (X[i+1] + Xhyperbolic[i+imin+1]) # ensures is trajectory
            # boundary conditions
            Y[-1][0] = np.dot(Pinv,X[l])[0] - q1 
            Y[-1][1] = np.dot(Pinv,X[l])[1] - q2
            Y[-1][2] = np.dot(Pinv,X[0])[2]
            Y[-1][3] = np.dot(Pinv,X[-1])[3]
            return Y.reshape(4*(imax - imin+1))

        def phi(y0,t0): # Evolves point forward by dt
            res = solve_ivp (f, [t0,dt+t0], y0 , method = "RK45" , max_step =dt/2 , rtol = 1e-12 )
            y = res .y. transpose()
            return  y[-1] 

        # We approximate the derivaitve of g
        Dg = np.zeros((4*(imax+1 - imin),4*(imax+1 - imin)))
        for i in range(0,(imax - imin)):
            Dg[4*i:4*i + 4, 4*i:4*i + 4] = Dphi
            Dg[4*i:4*i +4,4*i+4:4*i+8] = -np.eye(4)
        Dg[4*(imax+1 - imin)-4,4*(l):4*(l)+4] = np.dot(Pinv.transpose(),array([1,0,0,0]))
        Dg[4*(imax+1 - imin)-3,4*(l):4*(l)+4] = np.dot(Pinv.transpose(),array([0,1,0,0]))
        Dg[4*(imax+1 - imin)-2,0:4] = np.dot(Pinv.transpose(),array([0,0,1,0]))
        Dg[4*(imax+1 - imin)-1,4*(imax+1 - imin)-4:] = np.dot(Pinv.transpose(),array([0,0,0,1]))
        GP , GL, GU = scipy.linalg.lu(Dg) 
        def DG0solve(y0):
            x = np.transpose(GP).dot(y0)
            return Usolve(Lsolve(x,GL),GU)
        return g, DG0solve
    def findCenterPoint(q1,q2,t0index): # this function finds the point on the trajectory which satisfies the bvp with parameters q1,q2,t0
        imax, imin = min(t0index + Deltatplus, N-1), max(t0index - Deltatminus,0) # sets interval over which to do the bvp
        g, DG0solve = defineBVP(t0index,imax, imin)
        Xstart = lambda t : np.dot(P,np.array([exp(lambda2*t)*(q1*cos(omega*t) - q2 * sin(omega*t)) ,exp(lambda2*t)*(q1*sin(omega*t) + q2*cos(omega*t)) , t-t,t-t ])) # analytical solution to linear problem
        X = Xstart(thyperbolic[imin:imax+1]-thyperbolic[t0index]).transpose().reshape(4*(imax - imin + 1)) # intial guess for Newton method
        i = 0
        Gx = g(X,q1,q2)
        normGx = np.linalg.norm(Gx)
        epsG = 1e-12
        epsC = 1e-7
        delta = DG0solve(Gx)
        K = 250
        foundTrajectory = False
        while  (normGx > epsG or np.linalg.norm(delta) > epsC) and i < K:
            delta = DG0solve(Gx)
            X = X-delta # update sequence
            i += 1 # add counter
            Gx = g(X,q1,q2)
            normGx = np.linalg.norm(Gx) # compute norm to check the estimate
            if normGx > 100:
                break
        X = X.reshape((imax+1-imin,4)) + Xhyperbolic[imin:imax+1] 
        if (normGx < epsG and np.linalg.norm(delta) < epsC):
            foundTrajectory = True
        return X[ t0index - imin ],foundTrajectory
    t = thyperbolic[t0indices] # stores the times over which we solve the BVP
    pointsC = array([0,0,t[0]]) # will store the (q1,q2,t0) values
    valx = Xhyperbolic[t0indices[0],0] #  xvalues of points
    valy = Xhyperbolic[t0indices[0],1] # y values of points
    valvx = Xhyperbolic[t0indices[0],2] # vx values of points
    valvy =Xhyperbolic[t0indices[0],3] # vy values of points 
    timekey = np.zeros(len(t0indices)+1) # not used for centre trajectories
    unfoundTrajectoriesS = 0
    counter = 0
    numberofpoints = 1
    unfoundTrajectories =0
    # storing hyperbolic trajectory values.
    for l in range(0,len(t0indices)):
        if l > 0: # adding the hyperbolic trajectory, as this is contained in the stable manifold
            X = Xhyperbolic[t0indices[l]]
            pointsC = np.vstack((pointsC,array([0,0,t[l]])))
            valx = np.hstack((valx,X[0]))
            valy = np.hstack((valy,X[1]))
            valvx = np.hstack((valvx,X[2]))
            valvy = np.hstack((valvy,X[3]))
        for i in range(0, len(Q1)):
            for j in range(0,len(Q2)):
                X,foundTrajectory= findCenterPoint(Q1[i],Q2[j],t0indices[l]) # finding the trajectory
                counter +=1
                if counter%20 ==0:
                    print(str(100*counter/(len(t0indices)*M**2))+ "% trajectories calculated")
                    print("Trajectories not found: " +str(100*unfoundTrajectoriesS/counter) + "%")

                if foundTrajectory: # storing the values
                    pointsC = np.vstack((pointsC,array([Q1[i],Q2[j],t[l]])))
                    valx = np.hstack((valx,X[0]))
                    valy = np.hstack((valy,X[1]))
                    valvx = np.hstack((valvx,X[2]))
                    valvy = np.hstack((valvy,X[3]))
                    numberofpoints +=1
                else:
                    unfoundTrajectories +=1
        timekey[l+1] = numberofpoints
    print("Centre manifold algorithm failed to find: " + str(unfoundTrajectories)+ " out of " + str(M**2*len(t0indices)))
    return pointsC, valx,valy,valvx,valvy

In [4]:
#Parameters
R,M = 1.5,8  #  this controls Omega_C and the number of points take along it
Deltatminus = int(- N*np.log(10)/(2*T*lambda3))//1
Deltatplus = int(N*np.log(10)/(2*T*lambda4))
Q1 = np.linspace(-R,R,M)
Q2 = np.linspace(-R,R,M)
t0indices = np.arange(Deltatminus,N-1 -Deltatplus,5)
#t0indices = array([N//2,N//2+1])
CapsizeSide = "positive"

In [5]:
pointsCMplus, valxCplus,valyCplus,valvxCplus,valvyCplus = FindCentrePoints("positive")

1.1160714285714286% trajectories calculated
Trajectories not found: 0.0%
2.232142857142857% trajectories calculated
Trajectories not found: 0.0%
3.3482142857142856% trajectories calculated
Trajectories not found: 0.0%
4.464285714285714% trajectories calculated
Trajectories not found: 0.0%
5.580357142857143% trajectories calculated
Trajectories not found: 0.0%
6.696428571428571% trajectories calculated
Trajectories not found: 0.0%
7.8125% trajectories calculated
Trajectories not found: 0.0%
8.928571428571429% trajectories calculated
Trajectories not found: 0.0%
10.044642857142858% trajectories calculated
Trajectories not found: 0.0%
11.160714285714286% trajectories calculated
Trajectories not found: 0.0%
12.276785714285714% trajectories calculated
Trajectories not found: 0.0%
13.392857142857142% trajectories calculated
Trajectories not found: 0.0%
14.508928571428571% trajectories calculated
Trajectories not found: 0.0%
15.625% trajectories calculated
Trajectories not found: 0.0%
16.7410

In [6]:
np.save( 'pointsCMplus.npy',pointsCMplus )
np.save( 'valxCplus.npy',valxCplus )
np.save( 'valyCplus.npy', valyCplus)
np.save( 'valvxCplus.npy',valvxCplus )
np.save( 'valvyCplus.npy', valvyCplus)
np.save('RCplus.npy',array([R]))
np.save("tindicesCplus.npy",t0indices)#


In [7]:
pointsCMminus, valxCminus,valyCminus,valvxCminus,valvyCminus = FindCentrePoints("negative")

1.1160714285714286% trajectories calculated
Trajectories not found: 0.0%
2.232142857142857% trajectories calculated
Trajectories not found: 0.0%
3.3482142857142856% trajectories calculated
Trajectories not found: 0.0%
4.464285714285714% trajectories calculated
Trajectories not found: 0.0%
5.580357142857143% trajectories calculated
Trajectories not found: 0.0%
6.696428571428571% trajectories calculated
Trajectories not found: 0.0%
7.8125% trajectories calculated
Trajectories not found: 0.0%
8.928571428571429% trajectories calculated
Trajectories not found: 0.0%
10.044642857142858% trajectories calculated
Trajectories not found: 0.0%
11.160714285714286% trajectories calculated
Trajectories not found: 0.0%
12.276785714285714% trajectories calculated
Trajectories not found: 0.0%
13.392857142857142% trajectories calculated
Trajectories not found: 0.0%
14.508928571428571% trajectories calculated
Trajectories not found: 0.0%
15.625% trajectories calculated
Trajectories not found: 0.0%
16.7410

In [8]:
np.save( 'pointsCMminus.npy',pointsCMminus )
np.save( 'valxCminus.npy',valxCminus )
np.save( 'valyCminus.npy', valyCminus)
np.save( 'valvxCminus.npy',valvxCminus )
np.save( 'valvyCminus.npy', valvyCminus)
np.save('RCminus.npy',array([R]))
np.save("tindicesCminus.npy",t0indices)